In [8]:
import os
from keras.models import load_model
import pandas as pd

In [9]:
GPU = '1'
MODEL_FILE = 'out_1_5m/models/F2/e021-l=0.26689-vl=0.29136-a=0.91302-va=0.94067.h5'
INPUT_DIR = '/d2/caches/tf-speech/test/audio'
OUT_DIR = 'out_96'
OUT_CSV_FILE = 'predictions/predictions-f2@21.csv'
MSG_SIZE = (96,96)

In [10]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'

In [11]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

In [12]:
test_files_file = '%s/test_files.npy' % OUT_DIR
test_X_file = '%s/test_X.mem' % OUT_DIR

In [13]:
dg = DataGenerator(INPUT_DIR, extra_silence_dir=None)
dg.n_mels = MSG_SIZE[0]
dg.msg_w = MSG_SIZE[1]

In [14]:
model = load_model(MODEL_FILE)

In [15]:
if os.path.isfile(test_files_file):
    test_files = np.load(test_files_file)
else:
    test_files = dg.input_files[None]
    np.save(test_files_file, test_files)
    
print('test files: %d' % len(test_files))

test files: 158538


In [16]:
test_X = None

# generate msgs for test files
if not os.path.isfile(test_X_file):

    if os.path.isfile(test_X_file + '.tmp'):
        os.unlink(test_X_file + '.tmp')

    dg.debug = False
    test_X = np.memmap(
        test_X_file + '.tmp',
        np.float32,
        'w+',
        shape=(len(test_files), dg.n_mels, dg.msg_w, 1))

    for i, file in tqdm(enumerate(test_files), total=len(test_files)):
        wave, _ = dg.generate_audio(label='-', file=file, transform=False)
        msg = dg.msg(wave)
        msg = dg.normalize_msg(msg)
        test_X[i] = msg[:, :, np.newaxis]

    test_X.flush()
    os.rename(src=test_X_file + '.tmp', dst=test_X_file)

test_X = np.memmap(
    test_X_file,
    np.float32,
    'r',
    shape=(len(test_files), dg.n_mels, dg.msg_w, 1))

assert len(test_files) == len(test_X)
print('test files: %d' % len(test_X))

100%|██████████| 158538/158538 [1:17:55<00:00, 33.91it/s]


test files: 158538


In [10]:
predictions = {}

if test_X is None:
    # dynamically predict
    for file in tqdm(test_files):
        wave, _ = dg.generate_audio(label='-', file=file, transform=False)
        msg = dg.msg(wave)
        msg = dg.normalize_msg(msg)
        p = model.predict(msg[np.newaxis, :, :, np.newaxis])
        predictions[file] = p[0]
else:
    pp = model.predict(test_X, verbose=1, batch_size=500)
    # predict from test_X
    for i, file in enumerate(test_files):
        predictions[file] = pp[i]

predictions_df = pd.DataFrame.from_dict(predictions, orient='index')

158538/158538 [==============================] - 25s    


In [11]:
predictions_df.to_csv(OUT_CSV_FILE, index_label='file')